In [1]:
import pickle
import chess
import pandas as pd
import numpy as np 
from chess_minimax import get_minimax_move
from chess_random import get_random_move

In [2]:
with open('2014_08_over_2400_chess_data.pickle','rb') as read_file:
    chess_df = pickle.load(read_file)

chess_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 106 entries, Game47674 to Game993220
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   BlackElo       106 non-null    object
 1   WhiteElo       106 non-null    object
 2   moves          106 non-null    object
 3   move_num       106 non-null    object
 4   winner         106 non-null    object
 5   Black_Elo_Num  106 non-null    int64 
 6   White_Elo_Num  106 non-null    int64 
dtypes: int64(2), object(5)
memory usage: 6.6+ KB


In [3]:
chess_df
moves = chess_df['moves']

In [4]:
char_list = []
for i in range(1,500):
    char_list.append(str(i) + '.')

move_list = []

for game in moves:
    index = list(moves).index(game)
    allmoves = game.split()
    all_moves = [elem for elem in allmoves if elem not in char_list]
    board = chess.Board()
    for i in range(len(all_moves)):
        board.push_san(all_moves[i])
        fen = board.fen()
        row = [index, i, fen, all_moves[i]]
        move_list.append(row)

In [5]:
move_df = pd.DataFrame(move_list, columns = ['Game_Num', 'Move_Num', 'FEN', 'Move'])
move_df['Next_Move'] = move_df.Move.shift(-1)
ddf = move_df.groupby(['Game_Num'])['Move_Num', 'FEN', 'Move', 'Next_Move'].apply(lambda x : x[:-1])

<ipython-input-5-508f604a7da8>:3: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  ddf = move_df.groupby(['Game_Num'])['Move_Num', 'FEN', 'Move', 'Next_Move'].apply(lambda x : x[:-1])


In [6]:
#ddf['Rand_Move'] =ddf['FEN'].apply(get_random_move)
#ddf['Minimax_Move'] =ddf['FEN'].apply(get_minimax_move)

for index, row in ddf.iterrows():
    m0 = get_random_move(row['FEN'])
    ddf.loc[index,'Rand_Move'] = m0
    m1 = get_minimax_move(row['FEN'])
    ddf.loc[index,'Minimax_Move'] = m1
    print(index)
    
ddf.head()

(0, 0)
(0, 1)
(0, 2)
(0, 3)
(0, 4)
(0, 5)
(0, 6)
(0, 7)
(0, 8)
(0, 9)
(0, 10)
(0, 11)
(0, 12)
(0, 13)
(0, 14)
(0, 15)
(0, 16)
(0, 17)
(0, 18)
(0, 19)
(0, 20)
(0, 21)
(0, 22)
(0, 23)
(0, 24)
(0, 25)
(0, 26)
(0, 27)
(0, 28)
(0, 29)
(0, 30)
(0, 31)
(0, 32)
(0, 33)
(0, 34)
(0, 35)
(0, 36)
(0, 37)
(0, 38)
(0, 39)
(0, 40)
(0, 41)
(1, 43)
(1, 44)
(1, 45)
(1, 46)
(1, 47)
(1, 48)
(1, 49)
(1, 50)
(1, 51)
(1, 52)
(1, 53)
(1, 54)
(1, 55)
(1, 56)
(1, 57)
(1, 58)
(1, 59)
(1, 60)
(1, 61)
(1, 62)
(1, 63)
(1, 64)
(1, 65)
(1, 66)
(1, 67)
(1, 68)
(1, 69)
(1, 70)
(1, 71)
(1, 72)
(1, 73)
(1, 74)
(1, 75)
(1, 76)
(1, 77)
(1, 78)
(1, 79)
(1, 80)
(1, 81)
(1, 82)
(1, 83)
(1, 84)
(1, 85)
(1, 86)
(1, 87)
(1, 88)
(1, 89)
(1, 90)
(1, 91)
(1, 92)
(1, 93)
(1, 94)
(1, 95)
(1, 96)
(1, 97)
(1, 98)
(1, 99)
(1, 100)
(1, 101)
(1, 102)
(1, 103)
(1, 104)
(1, 105)
(1, 106)
(1, 107)
(1, 108)
(1, 109)
(1, 110)
(1, 111)
(1, 112)
(1, 113)
(1, 114)
(1, 115)
(1, 116)
(1, 117)
(1, 118)
(1, 119)
(1, 120)
(1, 121)
(1, 122)
(1, 123)
(1

KeyboardInterrupt: 

In [ ]:
ddf['Random_Compare'] = np.where(ddf['Next_Move'] == ddf['Rand_Move'], True, False)
ddf['Minimax_Compare'] = np.where(ddf['Next_Move'] == ddf['Minimax_Move'], True, False)

In [ ]:
ddf['Random_Compare'].value_counts()

In [ ]:
ddf['Minimax_Compare'].value_counts()

In [ ]:
ddf['Total_Moves'] = ddf.groupby(['Game_Num'])['Move_Num'].transform(max) + 1
ddf['Percent_Through_Game'] = (ddf['Move_Num']+1)/ddf['Total_Moves']
ddf['Stage_Of_Game'] = np.where(ddf['Percent_Through_Game'] < .3334, 'Beginning',
                       np.where(ddf['Percent_Through_Game'] < .6667, 'Middle',         
                               'End'))
ddf.head()

In [ ]:
pd.crosstab(ddf['Minimax_Compare'], ddf['Stage_Of_Game'])

In [ ]:
pd.crosstab(ddf['Random_Compare'], ddf['Stage_Of_Game'])

In [ ]:
#ddf['NeuralNet_Compare'].value_counts()

In [ ]:
ddf['Move_Num'].describe()

In [ ]:
import matplotlib.pyplot as plt

plt.hist(move_df['Move_Num'])

In [ ]:
test = chess_df[chess_df['move_num'] != 0]

In [ ]:
test.info()

In [ ]:
chess_df['Moves'] = chess_df['move_num'].astype(int)

In [ ]:
chess_df['Moves'].describe()